In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
data_list = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data_list.append(os.path.join(dirname, filename))
        print(data)
data = data_list[0]
data = pd.read_csv(data)
print(data.columns)
catagorical_cols = ['name', 'serving_size', 'ingredients', 'tag', 'allergy', 'description']
numerical_cols = ['energy', 'protein', 'total_fat', 'sat_fat', 'trans_fat', 'chol', 'carbs', 'total_sugar', 'added_sugar', 'sodium']
data.head()
# data.describe()

NameError: name 'data' is not defined

In [ ]:
for i in numerical_cols:
    print(data[i].dtype)
print('\n')
for i in catagorical_cols:
    print(data[i].dtype)

In [ ]:
manip_d = data.copy()
# serv = manip_d[manip_d['serving_size'].str.contains('g')]['serving_size']
# serv
numeric_serving_size = manip_d['serving_size'].str.extract('(\d+)').astype(float)

is_ml = manip_d['serving_size'].str.contains('ml')
grams_per_ml = 1.04
numeric_serving_size.loc[is_ml] *= grams_per_ml
numeric_serving_size = manip_d['serving_size'].str.extract('(\d+)').astype(float)
manip_d['serving_size'] = numeric_serving_size
# serv
manip_d = manip_d.drop('description', axis=1)
alergy_list = []
for i in manip_d['allergy'].unique():
    for j in i.split(','):
        if j not in alergy_list:
            alergy_list.append(j)
cdman_df = pd.get_dummies(manip_d['allergy'])
more_allergies = { 'index':range(42),
                  'Soya':[0 for _ in range(42)],
                  'Egg':[0 for _ in range(42)],
                  'Fish':[0 for _ in range(42)],
                  'Sulphites':[0 for _ in range(42)],
                  'Nuts':[0 for _ in range(42)]}
more_allergies = pd.DataFrame(more_allergies) 

coded_df = pd.concat([cdman_df,more_allergies,manip_d], axis=1)
coded_df.drop(['index'], axis=1, inplace=True)
coded_df.columns
# cdman_d.shape
# manip_d.shape
for index, row in coded_df.iterrows():
    if row['Cereal containing gluten, Milk, Egg, Fish, Soya'] == 1:
        coded_df.at[index,'Cereal containing gluten'] = 1
        coded_df.at[index,'Milk'] = 1
        coded_df.at[index,'Egg'] = 1
        coded_df.at[index,'Fish'] = 1
        coded_df.at[index,'Soya'] = 1
    elif row['Cereal containing gluten, Milk, Egg, Soya'] == 1:
        coded_df.at[index,'Cereal containing gluten'] = 1
        coded_df.at[index,'Milk'] = 1
        coded_df.at[index,'Egg'] = 1
        coded_df.at[index,'Soya'] = 1
    elif row['Cereal containing gluten, Milk, Nuts'] == 1:
        coded_df.at[index,'Cereal containing gluten'] = 1
        coded_df.at[index,'Milk'] = 1
        coded_df.at[index,'Nuts'] = 1
    elif row['Cereal containing gluten, Milk, Soya'] == 1:
        coded_df.at[index,'Cereal containing gluten'] = 1
        coded_df.at[index,'Milk'] = 1
        coded_df.at[index,'Soya'] = 1
    elif row['Cereal containing gluten, Milk, Soya, Sulphites'] == 1:
        coded_df.at[index,'Cereal containing gluten'] = 1
        coded_df.at[index,'Milk'] = 1
        coded_df.at[index,'Sulphites'] = 1
        coded_df.at[index,'Soya'] = 1
    elif row['Milk, Soya'] == 1:
        coded_df.at[index,'Milk'] = 1
        coded_df.at[index,'Soya'] = 1
    else:
        continue
delete_cols = ['Cereal containing gluten, Milk, Egg, Fish, Soya',
'Cereal containing gluten, Milk, Egg, Soya',
'Cereal containing gluten, Milk, Nuts',
'Cereal containing gluten, Milk, Soya', 
'Cereal containing gluten, Milk, Soya, Sulphites', 
'Milk, Soya']
coded_df.drop(delete_cols, axis=1, inplace=True)
coded_df
# alergy_list

In [ ]:
clone_df = coded_df.copy()
clone_df.drop(['tag'], axis=1, inplace=True)
clone_df.columns

In [ ]:
ingredients = []
# clone_df['ingredients']
for i in clone_df['ingredients']:
#     print(type(np.), i)
    if type(1.2)==type(i):
        i=np.nan
    else:
        
        for j in i.split(','):
            if j.strip() is not ingredients:
                ingredients.append(j.strip())
ingredients

In [ ]:
clone_df1 = clone_df.copy()
clone_df1.drop(['ingredients', 'allergy'], axis=1, inplace = True)
clone_df1 = clone_df1.iloc[:,-11:]
clone_df1

In [ ]:
def normalizer(dframe, col):
    for index, row in clone_df1.iterrows():
        val = (row[col]-dframe[col].mean())/(dframe[col].max()-dframe[col].mean())
        clone_df1.at[index, str(str(col)+'_rounded')] = np.around(val, decimals=2)
    dframe.drop([col],axis=1,inplace=True)
for i in clone_df1.columns:
    normalizer(clone_df1, i)
#     print(clone_df1.iloc[index, 6])
#     print(str(val))
clone_df1

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(12, 9), gridspec_kw={'height_ratios': [2, 1, 1]})
for index,value in enumerate(clone_df1.columns):
#     print(clone_df1[value])
    ax[index//4, index%4].hist(clone_df1[value], bins=30)
    ax[index//4, index%4].set_label(value)
plt.tight_layout()
plt.show()

In [ ]:
corr = clone_df1.corr()
sns.pairplot(corr)

In [ ]:
rel_tuple_list = []
accept_value = corr.mean().mean() + 0.4
col = corr.columns
for i in col:
    for j in col:
        if not i==j:
            if not (j,i) in rel_tuple_list:
                if np.absolute(corr.at[i,j]) > accept_value:
                    rel_tuple_list.append((i,j))
# len(rel_tuple_list)
rel_tuple_list

In [ ]:
important_corr = corr[['energy_rounded', 'protein_rounded', 'total_fat_rounded', 'sat_fat_rounded', 'chol_rounded', 'total_sugar_rounded', 'added_sugar_rounded', 'sodium_rounded']]

important_corr.drop(['serving_size_rounded', 'trans_fat_rounded', 'carbs_rounded'],axis=0,inplace=True)
important_corr

In [ ]:
fig, ax = plt.subplots(2,4,figsize=(6,8))
for index, value in enumerate(important_corr.columns):
    ax[index//4, index%4].bar(x=important_corr[value], height=value,color=['violet', 'pink', 'indigo', 'blue', 'green', 'yellow', 'orange', 'red'], width=0.04)